In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import json
import datetime
import time

In [ ]:
# prompt for Google API
def create_prompt(findings):
    """
    Creates a prompt for the Google Generative AI API based on the findings
    :param findings: The radiology findings to include in the prompt
    :return: The formatted prompt string
    """
    prompt = (
        "You are an experienced veterinary clinician. I will provide you with a list of radiology findings written by a veterinary radiologist. "
        "I will also provide a list of possible canine disease diagnoses. Mark each diagnosis Positive or Negative depending on your evaluation of the findings. "
        "Please provide a detailed explanation for each diagnosis classification. Adhere to the following guidlines when classifying the findings:\n"
        "Disease Diagnoses:\n"
        "perihilar_infiltrate: Mark Positive if there is evidence of infiltrate in the perihilar region of the lungs.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of infiltrate in the perihilar, or say the perihilar region or lungs are normal.\n"
        "pneumonia: Mark Positive if the findings contain evidence of pneumonia or the following key words: alveolar pattern.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of pneumonia, or say the lungs are normal.\n"
        "bronchitis: Mark Positive if the findings contain evidence of bronchitis or the following key words: bronchial pattern, "
        "bronchointerstitial pulmonary pattern, lobar bronchi narrowing, dynamic airway disease, bronchial wall thickening, "
        "peribronchial cuffing, bronchial markings, increased bronchovascular markings, airway thickening, or prominent bronchi in the lungs.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of bronchitis, or say the bronchi is normal.\n"
        "interstitial: Mark Positive if the findings contain evidence of interstitial pattern in the lungs or the following key words: "
        "interstitial pattern, interstitial pulmonary pattern, interstitial lung disease, interstitial markings, reticular pattern, "
        "diffuse interstitial opacity, interstitial thickening, ground-glass opacities, linear opacities.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of interstitial pattern, or say the lungs are normal.\n"
        "diseased_lungs: Mark Positive if the findings contain evidence of the presence of any lung-related disease, diseased lungs, or the following key words: "
        "diseased lungs, abnormal lung pattern, abnormal lung markings, abnormal lung opacity, abnormal lung structure, abnormal lung appearance, pulmonary disease, bronchointerstitial pattern.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of diseased lungs, or say the lungs are normal.\n"
        "hypo_plastic_trachea: Mark Positive if the findings contain evidence of hypoplastic trachea or the following key words: "
        "hypoplastic trachea, tracheal hypoplasia, tracheal stenosis, tracheal collapse, tracheal narrowing, tracheal deformity, tracheal malformation, small trachea diameter.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of hypoplastic trachea, or say the trachea is normal.\n"
        "cardiomegaly: Mark Positive if the findings contain evidence of cardiomegaly or the following key words: "
        "cardiomegaly, enlarged heart, heart enlargement, increased cardiac silhouette, heart size abnormality, heart volume increase, heart chamber enlargement.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of cardiomegaly, or say the heart is normal.\n"
        "pulmonary_nodules: Mark Positive if the findings contain evidence of pulmonary nodules or the following key words: "
        "pulmonary nodules, lung nodules, pulmonary masses, lung masses, pulmonary lesions, lung lesions, pulmonary opacities, lung opacities.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of pulmonary nodules, or say the lungs are normal.\n"
        "pleural_effusion: Mark Positive if the findings contain evidence of pleural effusion or the following key words: "
        "pleural effusion, fluid in the pleural space, pleural fluid accumulation, pleural fluid buildup, pleural fluid collection, pleural fluid presence, pleural fluid opacity.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of pleural effusion, or say the pleural space is normal.\n"
        "rtm: Mark Positive if the findings contain evidence or Redundant Tracheal Membrane (RTM) or the following key words: "
        "Redundant Tracheal Membrane, RTM, tracheal membrane redundancy, tracheal membrane laxity, tracheal membrane abnormality, tracheal membrane deformity, tracheal membrane malformation.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of RTM, or say the trachea is normal.\n"
        "focal_caudodorsal_lung: Mark Positive if the findings contain evidence of focal caudodorsal lung disease or the following key words: "
        "focal abnormalities in the caudodorsal lung region, focal caudodorsal lung disease, focal caudodorsal lung abnormalities, focal caudodorsal lung lesions, focal caudodorsal lung opacities, pulmonary bulla on caudal lung lobe.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of focal caudodorsal lung disease, or say the lungs are normal.\n"
        "focal_perihilar: Mark Positive if the findings contain evidence of focal perihilar lung disease or the following key words: "
        "focal abnormalities in the perihilar lung region, focal perihilar lung disease, focal perihilar lung abnormalities, focal perihilar lung lesions, focal perihilar lung opacities.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of focal perihilar lung disease, or say the lungs are normal.\n"
        "pulmonary_hypoinflation: Mark Positive if the findings contain evidence of pulmonary hypoinflation or the following key words: "
        "pulmonary hypoinflation, reduced lung volume, decreased lung inflation, hypoinflated lungs, underinflated lungs, hypoventilated lungs, pulmonary atelectasis.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of pulmonary hypoinflation, or say the lungs are normal.\n"
        "right_sided_cardiomegaly: Mark Positive if the findings contain evidence of right-sided cardiomegaly or the following key words: "
        "right-sided cardiomegaly, right heart enlargement, right ventricular enlargement, right atrial enlargement, right-sided heart disease, right-sided heart failure.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of right-sided cardiomegaly, or say the heart is normal.\n"
        "pericardial_effusion: Mark Positive if the findings contain evidence of pericardial effusion or the following key words: "
        "pericardial effusion, fluid in the pericardial sac, pericardial fluid accumulation, pericardial fluid buildup, pericardial fluid collection, pericardial fluid presence, pericardial fluid opacity, silhouette distortion in pericardial region.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of pericardial effusion, or say the pericardial sac is normal.\n"
        "bronchiectasis: Mark Positive if the findings contain evidence of bronchiectasis or the following key words: "
        "bronchiectasis, bronchial dilation, bronchial widening, bronchial distortion, bronchial deformity, bronchial malformation, bronchial wall thickening, bronchial cysts, dilated airways, widened airways.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of bronchiectasis, or say the bronchi are normal.\n"
        "pulmonary_vessel_enlargement: Mark Positive if the findings contain evidence of pulmonary vessel enlargement or the following key words: "
        "pulmonary vessel enlargement, enlarged pulmonary vessels, pulmonary artery enlargement, pulmonary vein enlargement, increased vascular markings, vascular congestion, vascular dilation.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of pulmonary vessel enlargement, or say the pulmonary vessels are normal.\n"
        "left_sided_cardiomegaly: Mark Positive if the findings contain evidence of left-sided cardiomegaly or the following key words: "
        "left-sided cardiomegaly, left heart enlargement, left ventricular enlargement, left atrial enlargement, left-sided heart disease, left-sided heart failure.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of left-sided cardiomegaly, or say the heart is normal.\n"
        "thoracic_lymphadenopathy: Mark Positive if the findings contain evidence of thoracic lymphadenopathy or the following key words: "
        "thoracic lymphadenopathy, intrathoracic lymphadenopathy, enlarged thoracic lymph nodes, mediastinal lymphadenopathy, hilar lymphadenopathy, thoracic lymph node enlargement, thoracic lymph node abnormalities.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of thoracic lymphadenopathy, or say the thoracic lymph nodes are normal.\n"
        "esophagitis: Mark Positive if the findings contain evidence of esophagitis or the following key words: "
        "esophagitis, inflammation of the esophagus, esophageal inflammation, esophageal irritation, esophageal swelling, esophageal redness, esophageal lesions, esophageal dilation, gas distention, esophageal foreign body.\n"
        "    Mark Negative if the findings have no mention of these keywords, do not contain evidence of esophagitis, or say the esophagus is normal.\n"
        "vhs_v2: Mark Positive if the findings list the Vertebral Heart Score (VHS) as greater than 10.5.\n"
        "    Mark Negative if the findings list the VHS as less than or equal to 10.5, or do not mention the VHS at all.\n"
        "IMPORTANT: Return the results in JSON format that matches the ground truth structure. "
        "Your response should contain ONLY the values for each disease classification (Positive, Negative, or Unknown). "
        "Do NOT include explanations in this JSON output.\n\n"
        
        "Reference the findings:\n" + findings + "\n\n"
        "CRITICAL: Return ONLY raw JSON - no markdown, no code blocks, no explanations, no extra text.\n"
        "Your response must start with { and end with } and be valid JSON.\n\n"
        "Return ONLY a JSON object in this EXACT JSON format (no markdown formatting):\n"
        "{\n"
        '  "perihilar_infiltrate": "Positive",\n'
        '  "pneumonia": "Negative",\n'
        '  "bronchitis": "Positive",\n'
        '  "interstitial": "Positive",\n'
        '  "diseased_lungs": "Positive",\n'
        '  "hypo_plastic_trachea": "Negative",\n'
        '  "cardiomegaly": "Negative",\n'
        '  "pulmonary_nodules": "Negative",\n'
        '  "pleural_effusion": "Negative",\n'
        '  "rtm": "Negative",\n'
        '  "focal_caudodorsal_lung": "Positive",\n'
        '  "focal_perihilar": "Negative",\n'
        '  "pulmonary_hypoinflation": "Negative",\n'
        '  "right_sided_cardiomegaly": "Negative",\n'
        '  "pericardial_effusion": "Negative",\n'
        '  "bronchiectasis": "Negative",\n'
        '  "pulmonary_vessel_enlargement": "Negative",\n'
        '  "left_sided_cardiomegaly": "Negative",\n'
        '  "thoracic_lymphadenopathy": "Negative",\n'
        '  "esophagitis": "Negative",\n'
        '  "vhs_v2": "Negative"\n'
        "}\n\n"

        "Each value must be exactly 'Positive', 'Negative', or 'Unknown'. "
        "Include all 21 disease classifications in the exact order shown above."
        "DO NOT include any other text, summaries, explanations, or formatting in your response.\n\n"
    )
    
    return prompt

In [ ]:
def read_excel_file(file_path):
    """
    Opens and reads input file Excel document
    :param file_path: Path to the input Excel file
    :return: DataFrame containing the contents of the Excel file
    """
    # check if file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Input file {file_path} does not exist")
    
    # check if file is an Excel file
    if not file_path.endswith(('.xls', '.xlsx')):
        raise ValueError(f"Input file {file_path} is not an Excel file")

    # try reading the Excel file and saving it as a DataFrame
    try:
        df = pd.read_excel(file_path)

    # raise a ValueError for any other exceptions that occur when reading the file 
    # with clearer error message
    except Exception as e:
        raise ValueError(f"Error reading Excel file {file_path}: {e}")
    return df

In [ ]:
def extract_ground_truth_scores(excel_file_path, output_filename=None):
    """
    Extract CaseID, Findings, and all 21 disease classifications from Excel file and convert to JSON format
    :param excel_file_path: Path to the Excel file
    :param output_filename: Optional custom filename for output JSON
    :return: DataFrame with all data, and output filename
    """
    
    try:
        # Read the Excel file
        df = read_excel_file(excel_file_path)
        print(f"Loaded Excel file with {df.shape[0]} rows and {df.shape[1]} columns")
        
        # Define the 21 disease columns in the correct order
        disease_columns = [
            'perihilar_infiltrate', 'pneumonia', 'bronchitis', 'interstitial', 'diseased_lungs',
            'hypo_plastic_trachea', 'cardiomegaly', 'pulmonary_nodules', 'pleural_effusion', 'rtm',
            'focal_caudodorsal_lung', 'focal_perihilar', 'pulmonary_hypoinflation', 'right_sided_cardiomegaly',
            'pericardial_effusion', 'bronchiectasis', 'pulmonary_vessel_enlargement', 'left_sided_cardiomegaly',
            'thoracic_lymphadenopathy', 'esophagitis', 'vhs_v2'
        ]
        
        # Check that all required columns exist
        required_columns = ['CaseID', 'Findings'] + disease_columns
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            print(f"Error: Missing columns: {missing_columns}")
            return None, None
        
        # Extract relevant columns and remove rows with no findings
        extracted_df = df[required_columns].copy()
        extracted_df = extracted_df.dropna(subset=['Findings'])  # Remove rows with no findings
        
        print(f"Extracted {len(extracted_df)} cases with valid findings and disease classifications")
        
        # Create JSON structure
        json_data = {
            "source_file": excel_file_path,
            "extraction_date": datetime.datetime.now().isoformat(),
            "total_cases": len(extracted_df),
            "disease_columns": disease_columns,
            "cases": []
        }
        
        # Add each case with CaseID, findings, and all disease classifications
        for index, row in extracted_df.iterrows():
            case_data = {
                "case_id": str(row['CaseID']).strip(),
                "findings": str(row['Findings']).strip()
            }
            
            # Add all disease classifications
            for disease in disease_columns:
                case_data[disease] = str(row[disease]).strip() if pd.notna(row[disease]) else "Unknown" # Positive/Negative value to str and removing whitespace (strip)
            
            json_data["cases"].append(case_data)
        
        # Generate output filename if not provided
        if output_filename is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            output_filename = f"ground_truth_scores_{timestamp}.json"
        
        # Write to JSON file
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2, ensure_ascii=False)
        
        print(f"✓ Successfully exported to: {output_filename}")
        
        # Show sample of extracted data
        print(f"\nSample of extracted data:")
        print(extracted_df[['CaseID', 'Findings'] + disease_columns[:5]].head(3))
        
        # Show disease distribution
        print(f"\nDisease classification summary:")
        for disease in disease_columns[:5]:  # Show first 5 diseases
            counts = extracted_df[disease].value_counts()
            print(f"  {disease}: {dict(counts)}")
        
        return extracted_df, output_filename
        
    except Exception as e:
        print(f"Error processing Excel file: {e}")
        return None, None

# Process your Excel file with disease classifications
excel_file_path = "/Users/Emily/radiology-project/CLEANED-Emily-canine_thorax_scoring.xlsx"
complete_data, complete_json_filename = extract_ground_truth_scores(excel_file_path)

In [ ]:
# Helper functions for confusion matrix

# Counts number of positive and negative cases for each disease in a JSON file
def count_pos_neg(json_file, disease):
    """
    Counts the number of positive and negative cases in the given JSON file for a given disease.
    :param json_file: JSON file containing case data, either ground truth or ai scores
    :param disease: Disease column to count positives and negatives for
    :return: Dictionary with counts of positive and negative cases
    """
    with open(json_file, 'r') as f:
        data = json.load(f)

    # extract disease_columns from the JSON data
    if disease not in data['disease_columns']:
        raise ValueError(f"JSON file does not contain '{disease}' key")

    # Initialize counters
    pos_count = 0
    neg_count = 0

    # Count positives and negatives
    for case in data['cases']:
        if case[disease] == 'Positive':
            pos_count += 1
        elif case[disease] == 'Negative':
            neg_count += 1

    # Return counts
    return {'Positive': pos_count, 'Negative': neg_count}
# makes sure that all ai file cases are included in ground truth file
def compare_check_files(ground_truth, ai_scores):
    """
    Compares the ground truth and AI scores JSON files for consistency.
    :param ground_truth: JSON file containing ground truth data
    :param ai_scores: JSON file containing AI scores data
    :return: None
    """
    # first, check if ai_scores has the same cases as ground_truth
    if len(ground_truth['cases']) >= len(ai_scores['cases']):
        raise ValueError("AI scores should have less than or equal to ground truth cases")
    # also make sure both have the same CaseIDs
    ground_truth_ids = {case['case_id'] for case in ground_truth['cases']}
    ai_scores_ids = {case['case_id'] for case in ai_scores['cases']}
    if ground_truth_ids != ai_scores_ids:
        raise ValueError("Ground truth and AI scores must have the same CaseIDs")

In [ ]:
# return Excel file, input file paths
def create_confusion_matrix(ground_truth, ai_scores):
    """
    Creates an Excel file containing confusion matrix from comparing ground truth and AI scores
    :param ground_truth: JSON file containing ground truth data
    :param ai_scores: JSON file containing AI scores data
    :return: confusion matrix export as Excel file
    """

    # compare the ground truth and ai_scores files
    #compare_check_files(ground_truth, ai_scores)

    # columns from Example Confusion Matrix
    cols = [
        'condition','tp_Positive','fn_Positive','tn_Positive','fp_Positive','Sensitivity','Specificity','Check',
        'Positive Ground Truth','Negative Ground Truth','Ground Truth Check'
    ]

    # create df to hold confusion matrix data
    df = pd.DataFrame(columns=cols)

   # Load JSON files if they are file paths (strings)
    if isinstance(ground_truth, str):
        with open(ground_truth, 'r') as f:
            ground_truth_data = json.load(f)
    else:
        ground_truth_data = ground_truth
        
    if isinstance(ai_scores, str):
        with open(ai_scores, 'r') as f:
            ai_scores_data = json.load(f)
    else:
        ai_scores_data = ai_scores
    
    # Get disease columns from the ground truth data
    disease_columns = ground_truth_data['disease_columns']
    
    # Process each disease (not 'condition' - that field doesn't exist)
    for disease in disease_columns:
        # Count ground truth cases for this disease
        gt_counts = count_pos_neg(ground_truth, disease)
        
        # Initialize confusion matrix counters
        tp = fp = tn = fn = 0
        
        # Calculate confusion matrix values for this disease
        for ai_case in ai_scores_data['cases']:
            # Find corresponding ground truth case
            gt_case = next((c for c in ground_truth_data['cases'] if c['case_id'] == ai_case['case_id']), None)
            if not gt_case:
                continue
                
            ai_pred = ai_case[disease]
            gt_true = gt_case[disease]
            
            if ai_pred == 'Positive' and gt_true == 'Positive':
                tp += 1
            elif ai_pred == 'Positive' and gt_true == 'Negative':
                fp += 1
            elif ai_pred == 'Negative' and gt_true == 'Positive':
                fn += 1
            elif ai_pred == 'Negative' and gt_true == 'Negative':
                tn += 1
        
        # Calculate metrics
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        # Create row for this disease
        row = {
            'condition': disease,
            'tp_Positive': tp,
            'fn_Positive': fn,
            'tn_Positive': tn,
            'fp_Positive': fp,
            'Sensitivity': sensitivity,
            'Specificity': specificity,
            'Check': tp + fn + tn + fp,
            'Positive Ground Truth': gt_counts['Positive'],
            'Negative Ground Truth': gt_counts['Negative'],
            'Ground Truth Check': gt_counts['Positive'] + gt_counts['Negative']
        }
        
        # Use pd.concat instead of deprecated df.append
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    
    # Export to Excel
    df.to_excel('confusion_matrix.xlsx', index=False)
    print("Confusion matrix created and saved as 'confusion_matrix.xlsx'")
    
    return df

In [ ]:
import google.generativeai as genai

def call_google_api(prompt, model='gemini-2.0-flash-lite', api_key=None):
    """
    Calls the Google Generative AI API with the given prompt
    :param prompt: The prompt to send to the Google Generative AI API
    :param model: The model to use for the API call
    :param api_key: The API key for authentication
    :return: The response from the Google Generative AI API
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model)

    generation_config = genai.types.GenerationConfig(
        temperature=0.2  # adjust creativity (0.0-2.0)
    )

    try:
        response = model.generate_content(prompt, generation_config=generation_config)
        response_text = response.text
        
        # Clean up markdown formatting if present
        if response_text.startswith('```json'):
            # Remove ```json from start and ``` from end
            response_text = response_text.replace('```json', '').replace('```', '').strip()
        elif response_text.startswith('```'):
            # Remove ``` from start and end
            response_text = response_text.replace('```', '').strip()
        if 'summary' in response_text or '**' in response_text or '***' in response_text:
            # Remove any markdown formatting or summary text
            response_text = response_text.replace('**', '').replace('***', '').replace('in summary', '').strip()
        # Remove any text or reading/trailing whitespace after the final closing brace
        if response_text.endswith('}'):
            response_text = response_text[:response_text.rfind('}') + 1].strip()
        
        return response_text
        
    except Exception as e:
        print(f"API Error: {e}")
        return f"Error: {e}"

In [ ]:
def ai_scorings(ground_truth_json_path, api_key, model_type="google", max_cases=None, output_filename=None):
    """
    Process cases and create AI predictions in the same format as ground truth scores
    :param ground_truth_json_path: Path to your ground truth JSON file with cases
    :param api_key: API key for AI service
    :param model_type: "google" or "openai"  
    :param max_cases: Optional limit on number of cases to process
    :param output_filename: Optional custom filename for AI predictions
    :return: AI predictions in ground truth format
    """
    
    print(f"🔍 ai_scorings function called at {datetime.datetime.now()}")

    try:
        # Read the ground truth JSON file
        with open(ground_truth_json_path, 'r', encoding='utf-8') as f:
            ground_truth_data = json.load(f)
        
        cases = ground_truth_data['cases']
        if max_cases:
            cases = cases[:max_cases]
        
        print(f"📊 Processing {len(cases)} cases with AI...")
        
        # Create AI predictions structure matching ground truth format
        ai_predictions = {
            "source_file": ground_truth_data['source_file'],
            "extraction_date": datetime.datetime.now().isoformat(),
            "total_cases": len(cases),
            "disease_columns": ground_truth_data['disease_columns'],
            "model_used": model_type,
            "cases": []
        }
        
        successful_predictions = 0
        failed_predictions = 0
        
        for i, case in enumerate(cases, 1):
            case_id = case['case_id']
            findings_text = case['findings']

            print(f"🔄 [AI_SCORINGS] Processing case {i}/{len(cases)} - CaseID: {case_id}")

            # Create prompt for this finding
            prompt = create_prompt(findings_text)
            
            # Get AI response
            print(f"📡 Making API call for case {case_id}")
            response = call_google_api(prompt, api_key=api_key)
            print(f"✅ API response received for case {case_id}")
            time.sleep(2)  # Sleep to avoid hitting API rate limits
            
            
            # Try to parse AI response as JSON
            try:
                ai_classifications = json.loads(response)
                
                # Create case data in ground truth format
                ai_case_data = {
                    "case_id": case_id,
                }
                
                # Add AI predictions for each disease
                for disease in ground_truth_data['disease_columns']:
                    if disease in ai_classifications:
                        ai_case_data[disease] = ai_classifications[disease]
                    else:
                        ai_case_data[disease] = "Unknown"  # Fallback if AI didn't provide this disease
                
                ai_predictions["cases"].append(ai_case_data)
                successful_predictions += 1
                print(f"✅ Successfully processed case {case_id}")

                
            except json.JSONDecodeError as e:
                print(f"  Warning: Failed to parse AI response for case {case_id}")
                print(f"  Error: {e}")
                print(response)  # Print the raw response for debugging
                
                # Create case with all Unknown values
                ai_case_data = {
                    "case_id": case_id,
                }
                
                # Set all diseases to Unknown for failed cases
                for disease in ground_truth_data['disease_columns']:
                    ai_case_data[disease] = "Unknown"
                
                ai_predictions["cases"].append(ai_case_data)
                failed_predictions += 1
        
        # Generate output filename if not provided
        if output_filename is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            model_name = model_type.lower()
            output_filename = f"ai_predictions_{model_name}_{timestamp}.json"
        
        # Write AI predictions to JSON file
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(ai_predictions, f, indent=2, ensure_ascii=False)
        
        print(f"\n✓ AI processing complete!")
        print(f"✓ Successful predictions: {successful_predictions}")
        print(f"✓ Failed predictions: {failed_predictions}")
        print(f"✓ AI predictions saved to: {output_filename}")
        
        return ai_predictions, output_filename
        
    except Exception as e:
        print(f"Error processing cases: {e}")
        return None, None



In [ ]:
my_api_key = "MY_API_KEY"  # Replace

In [127]:
ai_scores, ai_json_filename = ai_scorings(
    ground_truth_json_path=complete_json_filename,
    api_key=my_api_key,
    model_type="google",  # or "openai"
    max_cases=None,  # Process all cases
    output_filename=None  # Auto-generate filename
)

🔍 ai_scorings function called at 2025-08-03 23:13:13.607924
📊 Processing 50 cases with AI...
🔄 [AI_SCORINGS] Processing case 1/50 - CaseID: 2771776
📡 Making API call for case 2771776
API Error: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.0-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 200
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 46
}
]
✅ API response received for case 2771776
  Error: Expecting value: line 1 column 1 (char 0)
Error: 429 You exceeded your current quota, please 

KeyboardInterrupt: 

In [ ]:
create_confusion_matrix(ground_truth=complete_json_filename, ai_scores=ai_json_filename)